import time
plc=1
time.sleep(3000*plc)         #run after 1 houre = 3600 seconds

In [1]:
import os
import glob
import scipy
import random
import bisect
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import signal
from scipy.signal import find_peaks
import matplotlib.pyplot as plt
from preprocess import *                         #ماژول استخراج پنجره ها
from data import *                          #ماژول محلی ورود داده ها
from augment import *                            #ماژول های داده افزایی
from lstm_cnn import *                           #ماژول های داده افزایی با lstm_cnn
import augment
import importlib
importlib.reload(augment)
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import BatchNormalization,Input,ReLU,add,MaxPool1D,GlobalAveragePooling1D
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report,recall_score,precision_score
from tensorflow.keras import callbacks,Model,layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D,Conv1D,Dropout,MaxPooling1D,MaxPooling2D,Flatten,Dense
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import load_model
cwd = os.getcwd() #
fullscrn()

<h1><center><div style="direction:rtl;font-family:B Nazanin">Importing Data</div></center></h1>

In [2]:
rate=900
cls_num=5
btch=20
for i in range (1,cls_num+1):
    vars()['ecg'+str(i)],vars()['ecg_tst'+str(i)]=Worms0(i)

os.chdir(cwd)
clses_lens=np.array([])
ecg=np.array(ecg1)
print(1,len(vars()['ecg'+str(1)]), end='\t')
clses_lens=np.append(clses_lens,len(vars()['ecg'+str(1)]))
for i in range (1,cls_num):
    ecg=np.concatenate((ecg,vars()['ecg'+str(i)]),axis=0)
    clses_lens=np.append(clses_lens,len(vars()['ecg'+str(i)]))
    print(i,len(vars()['ecg'+str(i)]), end='\t')

mx_sig=max(clses_lens)

first_train_shape= (77, 901)
first_test_shape= (181, 901)
classes_quantity= 5
tr_lbls=	 {1, 2, 3, 4, 5}
Count_labels= [33 13 10 13  8]
max(train_feature_Altitude)= 5.0
min(train_feature_Altitude)= -4.8873
first_train_sample=
 [ 1.        -0.77859   -0.74444   -0.72546   -0.70269   -0.71408
 -0.72167   -0.73305   -0.71787   -0.73685   -0.76341   -0.77479
 -0.76341   -0.74823   -0.72546   -0.73305   -0.73305   -0.74064
 -0.74444   -0.76341   -0.7672    -0.76341   -0.75582   -0.74823
 -0.74823   -0.75582   -0.74823   -0.75962   -0.75203   -0.74823
 -0.74823   -0.73305   -0.73305   -0.74064   -0.771     -0.80895
 -0.83931   -0.86587   -0.87725   -0.87346   -0.919     -0.86207
 -0.86207   -0.85069   -0.82792   -0.85069   -0.78238   -0.7672
 -0.74444   -0.72546   -0.75203   -0.6951    -0.6951    -0.72167
 -0.72926   -0.74444   -0.74444   -0.79756   -0.75203   -0.67233
 -0.64577   -0.67613   -0.65716   -0.61541   -0.63818   -0.63439
 -0.59264   -0.61162   -0.5547    -0.42567   -0.30044   -0.2

1 33	1 33	2 13	3 10	4 13	

In [3]:
np.shape(ecg)

(102, 901)

In [4]:
ecg[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.])

<h2><center><div style="direction:rtl;font-family:B Nazanin">Base Train windows</div></center></h1>

In [5]:
smpl_rte=rate                                              # در ماژول ها نیز همین مقدار ثبت شده
i=0
windws=np.array([])

for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws'+str(cls)]=np.array(vars()['ecg'+str(cls)])  
    
print("each class and its windows =")

cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')

mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max =', mx_wndws)

each class and its windows =
1 33	2 13	3 10	4 13	5 8	
 max = 33


<h1><center><div style="direction:rtl;font-family:B Nazanin">Data Augmentation</div></center></h1>

In [6]:
smpl_rte=len(wndws1[0])                                        # در ماژول ها نیز همین مقدار ثبت شده

#add augmented data to base data
xtrain=np.empty((0,len(wndws1[0])), float) 
for i in range (1,cls_num+1):   
    xtrain=np.append(xtrain,vars()['wndws'+str(i)],axis=0)
    print('Up to class ', i, 'train shape = ', np.shape(xtrain))

Up to class  1 train shape =  (33, 901)
Up to class  2 train shape =  (46, 901)
Up to class  3 train shape =  (56, 901)
Up to class  4 train shape =  (69, 901)
Up to class  5 train shape =  (77, 901)


In [7]:
print(np.bincount(np.int16(xtrain[:,-1])))    #تعداد پنجره در هر کلاس از 0 تا 17

[ 0 33 13 10 13  8]


In [8]:
xtrain[:,-1]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 3., 3., 3., 3., 3.,
       3., 3., 3., 3., 3., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4., 4.,
       4., 5., 5., 5., 5., 5., 5., 5., 5.])

In [9]:
mx_aug=np.max(xtrain[:,:-1])
mx=np.max(ecg[:,:-1])
mn_aug=np.min(xtrain[:,:-1])
mn=np.min(ecg[:,:-1])

#for i in range (1,cls_num+1):             #نرمالسازی داده های افزایشی
xtrain[:,:-1]= 2*(xtrain[:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1

In [10]:
print('min trn =', np.min(xtrain[:,:-1]))
print('max trn =', np.max(xtrain[:,:-1]))

min trn = -1.0
max trn = 1.0


<h3><center><div style="direction:rtl;font-family:B Nazanin">Test Windows</div></center></h1>

In [11]:
wndws_test=np.empty((0,len(wndws1[0])), float)
i=0
windws=np.array([])
for cls in range (1,cls_num+1):                                 #ساخت پنجره های داده های آموزش اصلی
    vars()['wndws_tst'+str(cls)]=np.empty((0,len(wndws1[0])), float)
    vars()['wndws_tst'+str(cls)]=np.append(vars()['wndws_tst'+str(cls)],vars()['ecg_tst'+str(cls)],axis=0)   

In [12]:
#print(np.max(wndws_tst))
for i in range (1,cls_num+1):                # Normalization test windows
    cls=i
    print('\n cls', i, ' >> ')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))

    vars()['wndws_tst'+str(i)][:,:-1]= 2*(vars()['wndws_tst'+str(i)][:,:-1]-mn_aug)/(mx_aug - mn_aug) - 1
    
    print('after normalizing >>')
    print('max magnitude class', i , ' = ' ,np.max(vars()['wndws_tst'+str(cls)][:,:-1]))
    print('min magnitude class', i , ' = ' ,np.min(vars()['wndws_tst'+str(cls)][:,:-1]))
    
print("each class and its windows =")                    #تجمیع کلاس های تست
cls_wndws=np.array([])
for cls in range (1,cls_num+1):                                # آرایه ی تعداد پنجره ی هر کلاس
    wns=len(vars()['wndws_tst'+str(cls)])
    cls_wndws=np.append(cls_wndws,wns)
    print(cls, wns, end='\t')
    wndws_test=np.append(wndws_test,vars()['wndws_tst'+str(cls)],axis=0)
    
mx_wndws=int(np.max(cls_wndws))                        # حداکثر تعداد پنجره ی موجود بین کلاس ها
print('\n max instance in classes of test windows =', mx_wndws)


 cls 1  >> 
max magnitude class 1  =  4.1375
min magnitude class 1  =  -4.037800000000001
after normalizing >>
max magnitude class 1  =  0.9870973424048264
min magnitude class 1  =  -0.8129555012440277

 cls 2  >> 
max magnitude class 2  =  4.8591000000000015
min magnitude class 2  =  -4.3114
after normalizing >>
max magnitude class 2  =  1.1459805799590468
min magnitude class 2  =  -0.8731972609375345

 cls 3  >> 
max magnitude class 3  =  4.263200000000001
min magnitude class 3  =  -3.2160000000000006
after normalizing >>
max magnitude class 3  =  1.014774203492085
min magnitude class 3  =  -0.6320100402932824

 cls 4  >> 
max magnitude class 4  =  2.804300000000001
min magnitude class 4  =  -3.1806000000000005
after normalizing >>
max magnitude class 4  =  0.6935508730211157
min magnitude class 4  =  -0.6242156020873243

 cls 5  >> 
max magnitude class 5  =  2.7693
min magnitude class 5  =  -3.8073000000000006
after normalizing >>
max magnitude class 5  =  0.6858445075632471
min ma

In [13]:
print('min tst =', np.min(wndws_test[:,:-1]))
print('max tst =', np.max(wndws_test[:,:-1]))

min tst = -0.8731972609375345
max tst = 1.1459805799590468


In [14]:
len(wndws_test)

181

In [15]:
np.shape(wndws_test)

(181, 901)

In [16]:
Xtest=wndws_test[:,:-1]
ytest=np.int16(wndws_test[:,-1])

In [17]:
fullprint(ytest)

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5], dtype=int16)


<h3><center><div style="direction:rtl;font-family:B Nazanin">Train Windows</div></center></h1>

In [18]:
#ytraint=np.int32(np.array([ytrain]).T)
#trainx=np.concatenate((Xtrain, ytraint), axis=1)
trainx=np.random.permutation(xtrain)
Xtrain=np.array(trainx[:,:-1])
ytrain=np.int16(trainx[:,-1])
'''
ytestt=np.int32(np.array([ytest]).T)
testx=np.concatenate((Xtest, ytestt), axis=1)
testx=np.random.permutation(testx)
Xtest=np.int32(testx[:,:-1])
ytest=np.int32(testx[:,-1])'''

'\nytestt=np.int32(np.array([ytest]).T)\ntestx=np.concatenate((Xtest, ytestt), axis=1)\ntestx=np.random.permutation(testx)\nXtest=np.int32(testx[:,:-1])\nytest=np.int32(testx[:,-1])'

In [19]:
np.shape(wndws1)

(33, 901)

<h3><center><div style="direction:rtl;font-family:B Nazanin">Validation Windows</div></center></h1>

In [20]:
Xvalid=np.array(Xtrain)
yvalid=np.array(ytrain)

<h1><center><div style="direction:rtl;font-family:B Nazanin">CNN And UnBalanced RAW Data</div></center></h1>
<h1><center><div style="direction:rtl;font-family:Arial">Cross Entropy Loss Function</div></center></h1>

In [21]:
X_train=np.array(Xtrain)
y_train=np.array(ytrain)
X_test=np.array(Xtest)
y_test=np.array(ytest)
X_valid=np.array(Xtrain)
y_valid=np.array(ytrain)
print('X_train => ', X_train.shape)
print('y_train => ', y_train.shape)
print('X_test  => ', X_test.shape)
print('y_test  => ', y_test.shape)
print('X_valid  => ', X_valid.shape)
print('y_valid  => ', y_valid.shape)

X_train =>  (77, 900)
y_train =>  (77,)
X_test  =>  (181, 900)
y_test  =>  (181,)
X_valid  =>  (77, 900)
y_valid  =>  (77,)


In [22]:
if min(y_train)==0:
    print(' class was started from 0=Zero ')
else:
    print('!!!!!! class Should start from 0=Zero !!!!!!')
    print("!!!!!!   Reduce 1 From Class Number    !!!!!")
    y_train=np.array(y_train-1)
    y_test=np.array(y_test-1)
    y_valid=np.array(y_valid-1)
print('final Class Lables=',set(y_train))

!!!!!! class Should start from 0=Zero !!!!!!
!!!!!!   Reduce 1 From Class Number    !!!!!
final Class Lables= {0, 1, 2, 3, 4}


In [23]:
y_train_cat=to_categorical(y_train)
y_valid_cat=to_categorical(y_valid)
y_test_cat=to_categorical(y_test)


X_train = np.expand_dims(X_train, axis=2)
X_valid = np.expand_dims(X_valid, axis=2)
X_test = np.expand_dims(X_test, axis=2)
#y_train=np.transpose([y_train])

<h1><center><div style="direction:rtl;font-family:B Nazanin">Network</div></center></h1>

In [24]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=128, kernel_size=5, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=2048, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
4/4 [==============================] - 12s 148ms/step - loss: 4.6477 - accuracy: 0.2338 - val_loss: 1.5306 - val_accuracy: 0.4286
Epoch 2/1000
4/4 [==============================] - 0s 30ms/step - loss: 1.5206 - accuracy: 0.4286 - val_loss: 1.4749 - val_accuracy: 0.4286
Epoch 3/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.5613 - accuracy: 0.4286 - val_loss: 1.5262 - val_accuracy: 0.4286
Epoch 4/1000
4/4 [==============================] - 0s 30ms/step - loss: 1.5009 - accuracy: 0.4286 - val_loss: 1.4380 - val_accuracy: 0.4286
Epoch 5/1000
4/4 [==============================] - 2s 709ms/step - loss: 1.4772 - accuracy: 0.4286 - val_loss: 1.4424 - val_accuracy: 0.4286
Epoch 6/1000
4/4 [==============================] - 0s 30ms/step - loss: 1.4544 - accuracy: 0.4286 - val_loss: 1.4123 - val_accuracy: 0.4286
Epoch 7/1000
4/4 [==============================] - 0s 31ms/step - loss: 1.4455 - accuracy: 0.4416 - val_loss: 1.4293 - val_accuracy: 0.4286
Epoch 8/10

In [25]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 15ms/step
array([[52,  3,  1, 11,  9],
       [12, 16,  2,  1,  0],
       [10,  1,  7,  2,  5],
       [12,  0,  1, 15,  4],
       [ 1,  0,  3,  2, 11]], dtype=int64)
              precision    recall  f1-score   support

           0       0.60      0.68      0.64        76
           1       0.80      0.52      0.63        31
           2       0.50      0.28      0.36        25
           3       0.48      0.47      0.48        32
           4       0.38      0.65      0.48        17

    accuracy                           0.56       181
   macro avg       0.55      0.52      0.52       181
weighted avg       0.58      0.56      0.55       181



In [26]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=128, kernel_size=6, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=256, kernel_size=3, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=2048, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
4/4 [==============================] - 1s 111ms/step - loss: 3.6086 - accuracy: 0.2338 - val_loss: 1.5061 - val_accuracy: 0.4286
Epoch 2/1000
4/4 [==============================] - 2s 706ms/step - loss: 1.5967 - accuracy: 0.4286 - val_loss: 1.4846 - val_accuracy: 0.4286
Epoch 3/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.4898 - accuracy: 0.4286 - val_loss: 1.4858 - val_accuracy: 0.4286
Epoch 4/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.5174 - accuracy: 0.4286 - val_loss: 1.4488 - val_accuracy: 0.4286
Epoch 5/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.4995 - accuracy: 0.4286 - val_loss: 1.4278 - val_accuracy: 0.4286
Epoch 6/1000
4/4 [==============================] - 0s 28ms/step - loss: 1.4348 - accuracy: 0.4286 - val_loss: 1.4168 - val_accuracy: 0.4286
Epoch 7/1000
4/4 [==============================] - 0s 29ms/step - loss: 1.4059 - accuracy: 0.4286 - val_loss: 1.3766 - val_accuracy: 0.4286
Epoch 8/100

4/4 [==============================] - 0s 30ms/step - loss: 1.7144e-05 - accuracy: 1.0000 - val_loss: 1.0352e-05 - val_accuracy: 1.0000
Epoch 59/1000
4/4 [==============================] - 0s 30ms/step - loss: 1.0015e-05 - accuracy: 1.0000 - val_loss: 7.6974e-06 - val_accuracy: 1.0000
Epoch 60/1000
4/4 [==============================] - 0s 29ms/step - loss: 8.0534e-06 - accuracy: 1.0000 - val_loss: 5.9635e-06 - val_accuracy: 1.0000
Epoch 61/1000
4/4 [==============================] - 0s 29ms/step - loss: 5.8551e-06 - accuracy: 1.0000 - val_loss: 4.5609e-06 - val_accuracy: 1.0000
Epoch 62/1000
4/4 [==============================] - 0s 5ms/step - loss: 0.0233 - accuracy: 1.0000
np.shape(X_test)= (181, 900, 1)
Accuracy=  1.0


In [27]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 6ms/step
array([[52,  5,  2, 15,  2],
       [ 7, 19,  2,  3,  0],
       [10,  1,  5,  7,  2],
       [10,  0,  2, 14,  6],
       [ 1,  0,  1,  5, 10]], dtype=int64)
              precision    recall  f1-score   support

           0       0.65      0.68      0.67        76
           1       0.76      0.61      0.68        31
           2       0.42      0.20      0.27        25
           3       0.32      0.44      0.37        32
           4       0.50      0.59      0.54        17

    accuracy                           0.55       181
   macro avg       0.53      0.50      0.50       181
weighted avg       0.56      0.55      0.55       181



In [28]:
accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, btch
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data

model_crs = Sequential()

#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=24, activation='relu', input_shape=(n_timesteps,n_features)))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=12, activation='relu'))
#model_crs.add(Dropout(0.5))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
#model_crs.add(BatchNormalization())
model_crs.add(Conv1D(filters=2048, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''model_crs.add(Conv1D(filters=512, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
model_crs.add(Conv1D(filters=1024, kernel_size=3, activation='relu'))
model_crs.add(MaxPooling1D(pool_size=3))
'''
model_crs.add(Flatten())
#model_crs.add(Dropout(0.5))
model_crs.add(Dense(2000, activation='relu'))
model_crs.add(Dense(n_outputs, activation='softmax'))
model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 20, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])
# evaluate model_crs
_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))


print('Accuracy= ', accuracy)

#output = K.function([model_crs.layers[0].input],[model_crs.layers[3].output])
#output(X_test)[0]
#y_pred= model_crs.predict(X_test)                # Function 1

##    print('Round ', i, '==>', 'accuracy = ', accuracy)
##    acc_crs=np.append(acc_crs,accuracy)
#y_pred_crs = model_crs.predict_classes(X_test)   # Function 2

Epoch 1/1000
4/4 [==============================] - 6s 932ms/step - loss: 730.1962 - accuracy: 0.1948 - val_loss: 1.5251 - val_accuracy: 0.4286
Epoch 2/1000
4/4 [==============================] - 0s 72ms/step - loss: 4.3295 - accuracy: 0.2857 - val_loss: 1.5642 - val_accuracy: 0.4286
Epoch 3/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.8245 - accuracy: 0.3377 - val_loss: 1.5855 - val_accuracy: 0.4286
Epoch 4/1000
4/4 [==============================] - 0s 72ms/step - loss: 1.4770 - accuracy: 0.3377 - val_loss: 1.5825 - val_accuracy: 0.4286
Epoch 5/1000
4/4 [==============================] - 2s 759ms/step - loss: 1.2561 - accuracy: 0.4416 - val_loss: 1.5818 - val_accuracy: 0.4935
Epoch 6/1000
4/4 [==============================] - 0s 83ms/step - loss: 1.1361 - accuracy: 0.5974 - val_loss: 1.5770 - val_accuracy: 0.5455
Epoch 7/1000
4/4 [==============================] - 0s 71ms/step - loss: 1.2881 - accuracy: 0.4545 - val_loss: 1.5649 - val_accuracy: 0.5065
Epoch 8/1

In [29]:
#model_crs.summary()
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 2s 42ms/step
array([[57,  0, 18,  0,  1],
       [28,  0,  3,  0,  0],
       [10,  0, 13,  0,  2],
       [19,  0, 13,  0,  0],
       [ 4,  0,  7,  0,  6]], dtype=int64)
              precision    recall  f1-score   support

           0       0.48      0.75      0.59        76
           1       0.00      0.00      0.00        31
           2       0.24      0.52      0.33        25
           3       0.00      0.00      0.00        32
           4       0.67      0.35      0.46        17

    accuracy                           0.42       181
   macro avg       0.28      0.32      0.28       181
weighted avg       0.30      0.42      0.34       181



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [30]:
#VGG16_1D

accuracy=0
acc_crs=np.array([])
##for i in range (10):
##    if accuracy<0.2 :

verbose, epochs, batch_size = 1, 1000, 80
n_timesteps, n_features, n_outputs = X_train.shape[1], X_train.shape[2], y_train_cat.shape[1]
steps_per_epoch = len(X_train)//batch_size
validation_steps = len(X_valid)//batch_size # if you have test data



model_crs = Sequential()
#model_crs.add(Conv1D(input_shape=x_train.shape[1:],filters=64,kernel_size=9,padding="same", activation="relu"))
model_crs.add(Conv1D(filters=128, kernel_size=12, strides=1, activation='relu', input_shape=(n_timesteps,n_features))) #Replaced
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=3))

model_crs.add(Conv1D(filters=32, kernel_size=7, strides=1, padding="same", activation="relu"))
model_crs.add(BatchNormalization())
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=32, kernel_size=10, strides=1, padding="same", activation="relu"))
model_crs.add(Conv1D(filters=64, kernel_size=5, strides=2, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=256, kernel_size=15, strides=1, padding="same", activation="relu"))
model_crs.add(MaxPooling1D(pool_size=2,strides=2))

model_crs.add(Conv1D(filters=512, kernel_size=5, strides=1, padding="same", activation="relu"))
#model_crs.add(Conv1D(filters=128, kernel_size=3, strides=1, padding="same", activation="relu"))


model_crs.add(Flatten())
model_crs.add(Dropout(0.1))
model_crs.add(Dense(units=512,activation="relu"))
#model_crs.add(Dense(units=2048,activation="relu"))
model_crs.add(Dense(n_outputs, activation='softmax'))   #Replaced here from old Model

model_crs.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
# fit network   #CategoricalCrossentropy #sparse_categorical_crossentropy #SparseCategoricalCrossentropy

earlystopping = callbacks.EarlyStopping(monitor ="val_accuracy", mode ="max", patience = 50, restore_best_weights = True)

model_crs.fit(X_train, y_train_cat, epochs=epochs, batch_size=batch_size, verbose=verbose, validation_data = (X_valid, y_valid_cat), callbacks =[earlystopping])

_, accuracy = model_crs.evaluate(X_valid, y_valid_cat, batch_size=batch_size, verbose=1)

print('np.shape(X_test)=',np.shape(X_test))

print('Accuracy= ', accuracy)

#y_pred_vgg = model_crs.predict_classes(X_test)


Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 1.6682 - accuracy: 0.1299 - val_loss: 2.4279 - val_accuracy: 0.4286
Epoch 2/1000
1/1 [==============================] - 0s 57ms/step - loss: 62.2559 - accuracy: 0.4286 - val_loss: 1.6138 - val_accuracy: 0.1688
Epoch 3/1000
1/1 [==============================] - 0s 56ms/step - loss: 1.8213 - accuracy: 0.1688 - val_loss: 1.6085 - val_accuracy: 0.1688
Epoch 4/1000
1/1 [==============================] - 0s 45ms/step - loss: 1.6074 - accuracy: 0.1688 - val_loss: 1.5894 - val_accuracy: 0.4286
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 1.4652 - accuracy: 0.4286 - val_loss: 1.6083 - val_accuracy: 0.1688
Epoch 6/1000
1/1 [==============================] - 0s 49ms/step - loss: 1.5742 - accuracy: 0.1688 - val_loss: 1.6031 - val_accuracy: 0.4286
Epoch 7/1000
1/1 [==============================] - 0s 51ms/step - loss: 1.4582 - accuracy: 0.5714 - val_loss: 1.5937 - val_accuracy: 0.4286
Epoch 8/1000
1/1

Epoch 59/1000
1/1 [==============================] - 0s 45ms/step - loss: 6.9463e-04 - accuracy: 1.0000 - val_loss: 1.4384 - val_accuracy: 0.4286
Epoch 60/1000
1/1 [==============================] - 0s 48ms/step - loss: 6.8447e-04 - accuracy: 1.0000 - val_loss: 1.4372 - val_accuracy: 0.4286
Epoch 61/1000
1/1 [==============================] - 0s 45ms/step - loss: 5.9176e-04 - accuracy: 1.0000 - val_loss: 1.4382 - val_accuracy: 0.4286
Epoch 62/1000
1/1 [==============================] - 0s 45ms/step - loss: 5.2939e-04 - accuracy: 1.0000 - val_loss: 1.4359 - val_accuracy: 0.4286
Epoch 63/1000
1/1 [==============================] - 0s 46ms/step - loss: 4.9537e-04 - accuracy: 1.0000 - val_loss: 1.4375 - val_accuracy: 0.4286
Epoch 64/1000
1/1 [==============================] - 0s 44ms/step - loss: 4.8635e-04 - accuracy: 1.0000 - val_loss: 1.4387 - val_accuracy: 0.4286
Epoch 65/1000
1/1 [==============================] - 0s 47ms/step - loss: 4.0228e-04 - accuracy: 1.0000 - val_loss: 1.4372 -

In [31]:
predict_x=model_crs.predict(X_test)              # Function 1

y_pred_crs=np.argmax(predict_x,axis=1)                    # function from augment.py to remove 0 index predictions

#y_pred_crs = model_crs.predict_classes(X_test)

cm = confusion_matrix(y_test, y_pred_crs)
fullprint(cm)
acc3=accuracy_score(y_test, y_pred_crs)

print(classification_report(y_test, y_pred_crs))

#sns.heatmap(cm, annot=True)

#plt.imshow(cm)

6/6 [==============================] - 0s 21ms/step
array([[36, 20, 12,  8,  0],
       [ 7, 18,  6,  0,  0],
       [15,  1,  5,  4,  0],
       [24,  0,  6,  2,  0],
       [ 7,  4,  6,  0,  0]], dtype=int64)
              precision    recall  f1-score   support

           0       0.40      0.47      0.44        76
           1       0.42      0.58      0.49        31
           2       0.14      0.20      0.17        25
           3       0.14      0.06      0.09        32
           4       0.00      0.00      0.00        17

    accuracy                           0.34       181
   macro avg       0.22      0.26      0.24       181
weighted avg       0.29      0.34      0.30       181



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
